In [3]:
import pandas as pd

In [4]:
df_global = pd.read_csv("../output/global-indicators.csv")[["timestamp","rides_canceled","rides_not_accomplished","rides_completed","rides_accepted","total_rides"]]
df_specific = pd.read_csv("../output/specific-indicators.csv")[["timestamp","expected_total_time","total_time","expected_ride_price","ride_price"]]

In [5]:
df_global.head()

timestamp  rides_canceled  rides_not_accomplished  rides_completed  \
0        1.0               0                       0                0   
1        2.0               0                       0                0   
2        3.0               0                       0                0   
3        4.0               0                       0                0   
4        5.0               0                       0                0   

   rides_accepted  total_rides  
0               0            0  
1               0            0  
2               0            0  
3               0            0  
4               0            0

In [6]:
df_specific[113:117]

timestamp  expected_total_time  total_time  expected_ride_price  \
113     2633.0               146.75       626.0                19.07   
114     2633.0               112.33       318.0                14.92   
115     2635.0               149.25       414.0                17.31   
116     2651.0               169.58       307.0                15.38   

     ride_price  
113       41.82  
114       34.26  
115       39.45  
116       21.72

In [7]:
df_global['unserved_requests_100'] = 0
df_global['unserved_requests_200'] = 0
df_global['unserved_requests_500'] = 0
#df_global['unserved_requests_100_min'] = 0
#df_global['unserved_requests_200_min'] = 0
#df_global['unserved_requests_500_min'] = 0
#df_global['unserved_requests_100_max'] = 0
#df_global['unserved_requests_200_max'] = 0
#df_global['unserved_requests_500_max'] = 0
df_global['overhead_100'] = 0
df_global['overhead_200'] = 0
df_global['overhead_500'] = 0
#df_global['overhead_100_min'] = 0
#df_global['overhead_200_min'] = 0
#df_global['overhead_500_min'] = 0
#df_global['overhead_100_max'] = 0
#df_global['overhead_200_max'] = 0
#df_global['overhead_500_max'] = 0
df_global['price_fluctuation_100'] = 0
df_global['price_fluctuation_200'] = 0
df_global['price_fluctuation_500'] = 0

In [8]:
for i in range(100,5000):
    df_end_window = df_global.iloc[i-1,:]
    df_start_window = df_global.iloc[i-100,:]
    df_window_num = (df_end_window["rides_canceled"] - df_start_window["rides_canceled"]) + (df_end_window["rides_not_accomplished"] - df_start_window["rides_not_accomplished"])
    df_window_den = (df_end_window["total_rides"] + df_end_window["rides_accepted"]) - (df_start_window["total_rides"] + df_start_window["rides_accepted"])
    df_global.at[i-1,"unserved_requests_100"] = df_window_num / df_window_den if not df_window_den == 0 else 0

for i in range(200,5000):
    df_end_window = df_global.iloc[i-1,:]
    df_start_window = df_global.iloc[i-200,:]
    df_window_num = (df_end_window["rides_canceled"] - df_start_window["rides_canceled"]) + (df_end_window["rides_not_accomplished"] - df_start_window["rides_not_accomplished"])
    df_window_den = (df_end_window["total_rides"] + df_end_window["rides_accepted"]) - (df_start_window["total_rides"] + df_start_window["rides_accepted"])
    df_global.at[i-1,"unserved_requests_200"] = df_window_num / df_window_den if not df_window_den == 0 else 0

for i in range(500,5000):
    df_end_window = df_global.iloc[i-1,:]
    df_start_window = df_global.iloc[i-500,:]
    df_window_num = (df_end_window["rides_canceled"] - df_start_window["rides_canceled"]) + (df_end_window["rides_not_accomplished"] - df_start_window["rides_not_accomplished"])
    df_window_den = (df_end_window["total_rides"] + df_end_window["rides_accepted"]) - (df_start_window["total_rides"] + df_start_window["rides_accepted"])
    if df_window_num < 0:
        print(i)
        print(df_window_num)
    if df_window_den < 0:
        print(i)
        print(df_end_window)
    df_global.at[i-1,"unserved_requests_500"] = df_window_num / df_window_den if not df_window_den == 0 else 0

In [9]:
df_specific["overhead"] = df_specific.apply(lambda x: abs(x["total_time"]-x["expected_total_time"])/(x["expected_total_time"]), axis=1)
df_specific["price_fluctuation"] =  df_specific.apply(lambda x: abs(x["ride_price"]-x["expected_ride_price"])/(x["expected_ride_price"]), axis=1)
df_specific.head()

timestamp  expected_total_time  total_time  expected_ride_price  \
0      363.0               112.13       313.0                 9.32   
1      370.0               142.72       314.0                14.80   
2      394.0               154.37       226.0                16.84   
3      433.0               119.80       364.0                17.16   
4      517.0               143.68       453.0                20.32   

   ride_price  overhead  price_fluctuation  
0       16.74  1.791403           0.796137  
1       26.99  1.200112           0.823649  
2       22.55  0.464015           0.339074  
3       43.64  2.038397           1.543124  
4       50.26  2.152840           1.473425

In [10]:
for i in range(100,5000):
    df_filtered = df_specific[(float(i-100) <= df_specific["timestamp"]) & (df_specific["timestamp"] <= float(i))].reset_index()
    df_global.at[i-1,"overhead_100"] = df_filtered["overhead"].mean()
    df_global.at[i-1,'price_fluctuation_100'] = df_filtered["price_fluctuation"].mean()
for i in range(200,5000):
    df_filtered = df_specific[(float(i-200) <= df_specific["timestamp"]) & (df_specific["timestamp"] <= float(i))].reset_index()
    df_global.at[i-1,"overhead_200"] = df_filtered["overhead"].mean()
    df_global.at[i-1,'price_fluctuation_200'] = df_filtered["price_fluctuation"].mean()
for i in range(400,5000):
    df_filtered = df_specific[(float(i-500) <= df_specific["timestamp"]) & (df_specific["timestamp"] <= float(i))].reset_index()
    df_global.at[i-1,"overhead_500"] = df_filtered["overhead"].mean()
    df_global.at[i-1,'price_fluctuation_500'] = df_filtered["price_fluctuation"].mean()

In [11]:
df_global = df_global.fillna(0)

In [12]:
print(df_global.iloc[200,:])

timestamp                 201.000000
rides_canceled              1.000000
rides_not_accomplished      0.000000
rides_completed             0.000000
rides_accepted             14.000000
total_rides                 2.000000
unserved_requests_100       0.111111
unserved_requests_200       0.062500
unserved_requests_500       0.000000
overhead_100                0.000000
overhead_200                0.000000
overhead_500                0.000000
price_fluctuation_100       0.000000
price_fluctuation_200       0.000000
price_fluctuation_500       0.000000
Name: 200, dtype: float64


In [13]:
df_global.to_csv("../analysis/v5/flash_mob.csv", index=False)

In [588]:
unserved_100_mean = df_global["unserved_requests_100"].mean()
unserved_200_mean = df_global["unserved_requests_200"].mean()
unserved_500_mean = df_global["unserved_requests_500"].mean()
unserved_100_min = df_global["unserved_requests_100"].mean()
unserved_200_min = df_global["unserved_requests_200"].mean()
unserved_500_min = df_global["unserved_requests_500"].mean()
unserved_100_min = df_global["unserved_requests_100"].mean()
unserved_200_min = df_global["unserved_requests_200"].mean()
unserved_500_min = df_global["unserved_requests_500"].mean()

In [105]:
df_specific["overhead"] = df_specific.apply(lambda x: (x["total_time"]-x["expected_total_time"])/(x["expected_total_time"]), axis=1)
df_specific.head()

timestamp  expected_total_time  total_time  expected_ride_price  \
0      332.0               165.50       325.0                19.24   
1      366.0               171.42       272.0                19.64   
2      397.0               110.30       222.0                16.22   
3      407.0               183.67       387.0                18.96   
4      416.0               110.27       333.0                11.73   

   ride_price  overhead  price_fluctuation  
0       30.69  0.963746           0.963746  
1       30.49  0.586746           0.586746  
2       28.44  1.012693           1.012693  
3       37.26  1.107040           1.107040  
4       32.75  2.019860           2.019860

In [64]:
df_global.head()

timestamp  rides_canceled  rides_not_accomplished  rides_completed  \
0        1.0               0                       0                0   
1        2.0               0                       0                0   
2        3.0               0                       0                0   
3        4.0               0                       0                0   
4        5.0               0                       0                0   

   rides_accepted  total_rides  
0               0            0  
1               0            2  
2               0            2  
3               0            2  
4               0            2

In [ ]:
df_window = df_global.iloc[0:500,:]
df_window["unserved_numerator"] = df_window[]
df_window.head()

In [ ]:
df_499 = df_global.iloc[499,:]
df_499

In [ ]:
df_num = df_global.iloc[499,:]["rides_canceled"] - df_global.iloc[0,:]["rides_canceled"] +  df_global.iloc[499,:]["rides_not_accomplished"] - df_global.iloc[0,:]["rides_not_accomplished"]
df_num